In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider, IntSlider, VBox, Label
from scipy.optimize import minimize

# --- Funding factor ---
def compute_funding_factor(annual_rate_percent, period_fraction, compounding="8h", frequency_per_year=1095):
    r = annual_rate_percent / 100.0
    if compounding == "8h":
        return (1 + r / frequency_per_year) ** (frequency_per_year * period_fraction)
    else:
        return 1 + r * period_fraction

# --- Optimization logic ---
def compute_min_viable_loss(entry_short, discount, annual_rate, period_fraction, prices):
    entry_long = (1 - discount) * entry_short
    funding_factor = compute_funding_factor(annual_rate, period_fraction)

    def total_pnl_vector(w_short):
        w_long = 1 - w_short
        short_raw_pnl = np.maximum(entry_short - prices, 0) * w_short
        funding_cost = (funding_factor - 1) * entry_short * w_short
        short_pnl = short_raw_pnl + funding_cost
        long_pnl = (prices - entry_long) * w_long
        return short_pnl + long_pnl

    def find_min_loss(tol=1e-3, max_iter=40):
        low = 0.0
        high = entry_short
        optimal_weight = np.nan

        for _ in range(max_iter):
            mid = (low + high) / 2

            def constraint(w):
                return np.min(total_pnl_vector(w[0])) + mid

            result = minimize(
                lambda w: w[0],
                x0=[0.5],
                bounds=[(0, 1)],
                constraints={"type": "ineq", "fun": constraint},
                method='SLSQP',
                options={"maxiter": 300, "ftol": 1e-8, "disp": False}
            )

            if result.success:
                optimal_weight = result.x[0]
                high = mid
            else:
                low = mid

            if high - low < tol:
                break

        return high / entry_short, optimal_weight

    return find_min_loss()

# --- Interactive plot function ---
def interactive_hedge(entry_short=150, discount=0.2, annual_rate=0, months=34):
    period_fraction = months / 12
    prices = np.linspace(0, entry_short * 3, 5000)

    max_loss_pct, short_weight = compute_min_viable_loss(
        entry_short, discount, annual_rate, period_fraction, prices)

    entry_long = (1 - discount) * entry_short
    funding_factor = compute_funding_factor(annual_rate, period_fraction)

    w_long = 1 - short_weight
    short_pnl = np.maximum(entry_short - prices, 0) * short_weight + (funding_factor - 1) * entry_short * short_weight
    long_pnl = (prices - entry_long) * w_long
    total_pnl = short_pnl + long_pnl

    plt.figure(figsize=(10, 6))
    plt.plot(prices, total_pnl, label="Total PnL", linewidth=2)
    plt.plot(prices, short_pnl, '--', label="Short PnL")
    plt.plot(prices, long_pnl, ':', label="Long PnL")
    plt.axhline(-max_loss_pct * entry_short, color='red', linestyle=':', label=f"Max Total Loss ({max_loss_pct:.2%})")
    plt.axhline(0, color='black', linewidth=0.5)
    plt.title(f"PnL Curve | Max Loss: -{max_loss_pct:.2%}, Optimal Short Weight: {short_weight:.3f}")
    plt.xlabel("Price")
    plt.ylabel("PnL")
    plt.grid(True)
    plt.legend()
    plt.show()

# --- Display with interactive widgets ---
entry_slider = FloatSlider(value=150, min=10, max=500, step=10, description='Entry Price')
discount_slider = FloatSlider(value=0.4, min=0.05, max=0.95, step=0.05, description='Discount')
funding_slider = IntSlider(value=-20, min=-50, max=50, step=5, description='Funding %')
months_slider = IntSlider(value=22, min=1, max=36, step=1, description='Months')

ui = VBox([
    Label("🎛️ Adjust parameters below:")
])

out = interactive(interactive_hedge, entry_short=entry_slider, discount=discount_slider, annual_rate=funding_slider, months=months_slider)
display(ui, out)

interactive(children=(FloatSlider(value=150.0, description='Entry Price', max=500.0, min=10.0, step=10.0), Flo…